[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/04_Baseline_and_Metrics.ipynb)


In [1]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


PosixPath('/home/runner/work/IINTS-SDK/IINTS-SDK')

# Baseline Comparison and Metrics

Compare your algorithm to the built-in baseline and compute clinical metrics.


In [2]:
import json
from iints.presets import load_presets

import iints
from iints.core.algorithms.mock_algorithms import ConstantDoseAlgorithm
from iints.metrics import calculate_full_metrics

presets = load_presets()
preset = next(p for p in presets if p["name"] == "baseline_t1d")

outputs = iints.run_simulation(
    algorithm=ConstantDoseAlgorithm,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=21,
    output_dir="results/baseline_metrics",
    compare_baselines=True,
)

comparison = outputs["baseline_comparison"]
comparison


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


{'reference': 'Standard PID',
 'rows': [{'algorithm': 'ConstantDoseAlgorithm',
   'tir_70_180': np.float64(35.714285714285715),
   'tir_below_70': np.float64(64.28571428571429),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 13,
   'total_violations': 13},
  {'algorithm': 'Standard PID',
   'tir_70_180': np.float64(38.46153846153847),
   'tir_below_70': np.float64(61.53846153846154),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 1,
   'total_violations': 13},
  {'algorithm': 'Standard Pump',
   'tir_70_180': np.float64(35.714285714285715),
   'tir_below_70': np.float64(64.28571428571429),
   'tir_above_180': np.float64(0.0),
   'bolus_interventions': 13,
   'total_violations': 13}]}

In [3]:
glucose = outputs["results"]["glucose_actual_mgdl"]
calculate_full_metrics(glucose)


ClinicalMetricsResult(tir_70_180=np.float64(35.714285714285715), tir_70_140=np.float64(35.714285714285715), tir_70_110=np.float64(21.428571428571427), tir_below_70=np.float64(64.28571428571429), tir_below_54=np.float64(57.14285714285714), tir_above_180=np.float64(0.0), tir_above_250=np.float64(0.0), cv=np.float64(66.2120626678778), sd=39.221828870372676, gmi=np.float64(4.726941428460387), mean_glucose=59.2366817918222, median_glucose=47.800038059461805, hi=np.float64(32.54095982687531), lbgi=np.float64(3.8652858895040367), hbgi=np.float64(0.012658994988496425), readings_per_day=288.0, data_coverage=100.0)